# Todo list: 
<li> Find a way to "trigger" messege to kafka (May not need to store in hdfs anymore-> contr)</li>
<li> Push to github </li>

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.11/2.4.1/spark-sql-kafka-0-10_2.11-2.4.1.jar

In [ ]:
!hdfs dfs -rm -r -skipTrash /user/ysunthor/y_checkpoint

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.types import StructType
import json
emailSchema = StructType() \
        .add("subject", StringType()) \
        .add("body", StringType())

# Group the data by window and subject and compute the count of each group

<li>The engine will maintain state and allow late data to be update the state until (max event time seen by the engine - late threshold > 1 minute). In other words, late data within the threshold will be aggregated, but data later than the threshold will be dropped.</li> 
<li>Count subject within 5 minutes windows, updating every 5 sec</li>
<li>Grouping is 2 min delay on Spark</li>

In [ ]:

# Subscribe to 1 topic

# .option("kafka.bootstrap.servers", "188.185.79.229:9092")\
# .option("startingOffsets", "earliest") \

raw_data = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "188.185.79.229:9092")\
.option("subscribe","mock_email_alert")\
.load()\
.select(
    from_json(col("value").cast("string"), emailSchema).getField("subject").alias("subject"),
    col("timestamp").alias("timestamp"))\

raw_data.printSchema()

## Pattern 
<li>Similar Subject name more than N number per window in "filtered_email"</li>

<li>Different Subject name more than N number per window in "strenge_email"</li>

In [ ]:
groupped_data=raw_data\
.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "1 minute", "1 minute"),"subject")\
.agg(count("subject"))\

groupped_data.printSchema()

In [ ]:
unique_row=raw_data\
.select(
    window("timestamp", "1 minute"),
    "timestamp",
    "subject"
  )\
.withWatermark("window", "1 day")\
.dropDuplicates(["subject", "window"])\

In [ ]:
from pyspark.sql.functions import concat_ws, collect_list
seq=raw_data\
.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "1 minute", "1 minute"))\
.agg(concat_ws(',',collect_list("subject")).alias("subject_sequence"))

In [ ]:
filtered_data= groupped_data.filter("count(subject) > 10")
filtered_data.printSchema()

In [ ]:
streamout_data_flow=filtered_data\
  .selectExpr("CAST(window AS STRING) AS key", "to_json(struct(subject,count)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "188.185.79.229:9092") \
  .option("topic", "filter_alert") \
  .option("checkpointLocation", "/cms/users/carizapo/ming/kafka") \
    .outputMode("Append")\
    .start()

In [ ]:
hdfs_data_flow=groupped_data.writeStream \
.outputMode("append")\
.format("parquet")\
 .option("path", "/cms/users/carizapo/ming/groupdata") \
 .option("checkpointLocation", "/cms/users/carizapo/ming/checkpoint") \
 .outputMode("append") \
 .start()

In [ ]:
raw_data_flow = raw_data.writeStream.queryName("email").outputMode("Append").format("memory").start()
groupped_data_flow = groupped_data.writeStream.queryName("groupped_email").outputMode("Append").format("memory").start()
# filtered_data_flow = filtered_data.writeStream.queryName("filtered_email").outputMode("Append").format("memory").start()
# unique_row_flow=unique_row.writeStream.queryName("unique_email").outputMode("Append").format("memory").start()
# seq_flow=seq.writeStream.queryName("seq_email").outputMode("Append").format("memory").start()""

In [ ]:
spark.streams.active

In [ ]:
# raw_data.isStreaming
# unique_row.isStreaming
# filtered_data.isStreaming
# groupped_data.isStreaming

In [ ]:
# streamout_data_flow.processAllAvailable()
# hdfs_data_flow.processAllAvailable()

In [ ]:
# raw_data_flow.lastProgress
# groupped_data_flow.lastProgress
# unique_row_flow.lastProgress
# filtered_data_flow.lastProgress
# streamout_data_flow.lastProgress

In [ ]:
raw_data_flow.stop()
groupped_data_flow.stop()
# filtered_data_flow.stop()
hdfs_data_flow.stop()
# streamout_data_flow.stop()
# unique_row_flow.stop()
# seq_flow.stop()

In [ ]:
alerts = spark.sql("select * from email")
alerts.show()

In [ ]:
alerts = spark.sql("select * from groupped_email")
alerts.show()

In [ ]:
alerts = spark.sql("select * from filtered_email")
alerts.show()

In [ ]:
alerts = spark.sql("select * from unique_email")
alerts.show()

In [ ]:
alerts = spark.sql("select * from seq_email")
alerts.show()